In [1]:
!pip install -q peft datasets
!pip install -q accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.1 MB/s eta 0:00:00


In [2]:
!pip install -q transformers

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup ,TrainingArguments
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import DataCollatorWithPadding

In [4]:
device = "cuda"
model_name_or_path = "databricks/dolly-v2-3b"
tokenizer_name_or_path = "databricks/dolly-v2-3b"

text_column = "prompt"
label_column = "label"

In [5]:
from datasets import load_from_disk

# Specify the directory where the dataset is saved
saved_directory = "/content/drive/MyDrive/Colab Notebooks/Table to insights/Data/Analytical Datset"

# Load the dataset from the specified directory
dataset = load_from_disk(saved_directory)


In [6]:
dataset['train'].features

{'table': [{'cpu_usage': Value(dtype='float64', id=None),
   'date': Value(dtype='string', id=None),
   'diskio_usage': Value(dtype='float64', id=None),
   'ram_usage': Value(dtype='float64', id=None)}],
 'prompt': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None)}

# Trainer API

In [7]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

In [8]:
def preprocess_data(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs,padding="max_length",max_length = 2000,truncation=True, return_tensors="pt")
    labels = tokenizer(targets, padding="max_length", max_length = 256,truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

In [9]:
processed_datasets = dataset.map(
    preprocess_data,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/516 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/129 [00:00<?, ? examples/s]

In [10]:
processed_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 516
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 129
    })
})

In [11]:
train_dataset = processed_datasets['train']
validation_dataset = processed_datasets['validation']

In [12]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,load_in_8bit=True,device_map='auto')

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

In [13]:
from peft import prepare_model_for_int8_training

In [14]:
model.train()
model.resize_token_embeddings(len(tokenizer))
def create_peft_config(model):

    peft_config = PrefixTuningConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        num_virtual_tokens=20
    )

    # prepare int-8 model for training
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

model, peft_config = create_peft_config(model)

trainable params: 3,276,800 || all params: 2,778,362,880 || trainable%: 0.11793995750475907


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:141: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [15]:
output_dir = "/content/drive/MyDrive/Colab Notebooks/Table to insights/dolly"

In [16]:
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_hf",
    learning_rate = 1e-4,
    num_train_epochs=1,
    gradient_accumulation_steps= 1,
    per_device_train_batch_size= 1,
    gradient_checkpointing=False
)

# Auto trainer

In [17]:
from transformers import Trainer

In [18]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        # data_collator=default_data_collator,
    )

In [19]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float3

RuntimeError: ignored

In [20]:
!nvidia-smi

Thu Dec  7 08:39:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    27W /  70W |   4983MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------